In [0]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive" 

# **LOADING PACKAGES**

In [0]:
!pip install geopandas
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cython
!pip install contextily
import math
import pandas as pd
from datetime import datetime, timezone
import geopandas as gpd
from shapely.geometry import Point
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import contextily as ctx
import timeit

# **PREPROCESSING: CONVERT TEMPORAL DATA TO L-SPACE EDGE- LIST OF  SELECTED TIMESPAN**


Rome 8AM - 9AM 2017-11-06

In [0]:
"""
The code below converts the full-day network temporal event list to an L-space
edge list of the selected timespan (either 8AM-9AM or 11-12AM our study).
An example of the resulting edge list is visible in Table 3 of the paper.
"""


combined_L =  pd.read_csv(DATA_PATH + "/rome/network_combined.csv", sep = ";") #Loading the combined edge list as introduced by Kujala et al. (2018)
data = pd.read_csv(DATA_PATH + "/rome/network_temporal_day.csv", sep = ";") #Loading the network event list as introduced by Kujala et al. (2018)
data["dep_time_ut"] = pd.to_datetime(data["dep_time_ut"], unit = "s") #transforms Unix timestamp to datetime
data["arr_time_ut"] = pd.to_datetime(data["arr_time_ut"], unit = "s") #ransforms Unix timestamp to  datetime
data["dep_time_ut"] += pd.to_timedelta(1, unit='h') #UTC+1h  https://www.timeanddate.com/time/zone/italy/rome
data["arr_time_ut"] += pd.to_timedelta(1, unit='h') #UTC+1h  https://www.timeanddate.com/time/zone/italy/rome
data["travel_time"] = data["arr_time_ut"]-data["dep_time_ut"] #calculate total travel time
data = data[(data["dep_time_ut"] >= "2017-11-06 08:00:00") & (data["dep_time_ut"] < "2017-11-06 09:00:00")] #selecting timespan: 8am-9am
data["travel_time"] = data["travel_time"] / np.timedelta64(1, 's') #converting timedelta to seconds
data = data.groupby(['from_stop_I', 'to_stop_I', "route_type","route_I", "travel_time"]).size()
data = data.reset_index(name="n_vehicles") #Put indexes back
data["travel_time"] = data["travel_time"].astype(int) #travel time value as integer
data["n_vehicles"] = data["n_vehicles"].astype(int) #number of vehicles values as integer
data.loc[:,["travel_time"]]=data.loc[:,["travel_time"]].values*data.loc[:,["n_vehicles"]].values #travel time*n_vehicles to get the right value after the groupby function
data = data.groupby(['from_stop_I', 'to_stop_I', "route_type", "route_I"]).agg({"travel_time": "sum",
                                                                    "n_vehicles": "sum"}).reset_index() #merge mostly duplicate rows (i.e. same edges in the network) 
data.loc[:,["travel_time"]]=data.loc[:,["travel_time"]].values/data.loc[:,["n_vehicles"]].values  #travel time/n_vehicles to derive the right values out of the groupby function                                                          
data['route_I_counts'] = data[["route_I", 'n_vehicles']].astype(str).apply(lambda x: ':'.join(x), axis=1) #create the route_I_counts column  
data = data.drop(columns = ["route_I"]) #drop route_I
data.loc[:,["travel_time"]] = data.loc[:,["travel_time"]].values*data.loc[:,["n_vehicles"]].values #travel time*n_vehicles to get the right value after the groupby function
data = data.groupby(['from_stop_I', 'to_stop_I', "route_type"]).agg({"travel_time": "sum",
                                                                    "n_vehicles": "sum",
                                                                    'route_I_counts': ','.join}).reset_index()
data.loc[:,["travel_time"]]=data.loc[:,["travel_time"]].values/data.loc[:,["n_vehicles"]].values #travel time/n_vehicles to derive the right values out of the groupby function

for row in data.itertuples(): #adding the "d" values from the combined edge list to the newly developed temporal event list 
  for row1 in combined_L.itertuples():
    if row[1] == row1[1] and row[2] == row1[2]:
      data.at[row.Index, "d"] = row1[3]
      
data = data.rename(columns = {"travel_time": "duration_avg"}) #rename column
data = data[["to_stop_I", "from_stop_I", "d", "duration_avg", "n_vehicles", "route_I_counts", "route_type"]] #Columns back in the right order
data.to_csv(DATA_PATH + "/rome/combined_L_8am_9am.csv", index = False)

Rome 11AM-12AM 2017-11-06

In [0]:
"""
The code below converts the full-day network temporal event list to an L-space
edge list of the selected timespan (either 8AM-9AM or 11-12AM our study).
An example of the resulting edge list is visible in Table 3 of the paper.
"""

combined_L =  pd.read_csv(DATA_PATH + "/rome/network_combined.csv", sep = ";") #Loading the combined edge list as introduced by Kujala et al. (2018)
data = pd.read_csv(DATA_PATH + "/rome/network_temporal_day.csv", sep = ";")  #Loading the full-day network event list as introduced by Kujala et al. (2018)
data["dep_time_ut"] = pd.to_datetime(data["dep_time_ut"], unit = "s") #transforms Unix timestamp to datetime
data["arr_time_ut"] = pd.to_datetime(data["arr_time_ut"], unit = "s") #ransforms Unix timestamp to  datetime
data["dep_time_ut"] += pd.to_timedelta(1, unit='h') #UTC+1h  https://www.timeanddate.com/time/zone/italy/rome
data["arr_time_ut"] += pd.to_timedelta(1, unit='h') #UTC+1h  https://www.timeanddate.com/time/zone/italy/rome
data["travel_time"] = data["arr_time_ut"]-data["dep_time_ut"] #calculate total travel time
data = data[(data["dep_time_ut"] >= "2017-11-06 11:00:00") & (data["dep_time_ut"] < "2017-11-06 12:00:00")] #selecting timespan: 8am-9am
data["travel_time"] = data["travel_time"] / np.timedelta64(1, 's') #converting timedelta to seconds
data = data.groupby(['from_stop_I', 'to_stop_I', "route_type","route_I", "travel_time"]).size()
data = data.reset_index(name="n_vehicles") #Put indexes back
data["travel_time"] = data["travel_time"].astype(int) #travel time value as integer
data["n_vehicles"] = data["n_vehicles"].astype(int) #number of vehicles values as integer
data.loc[:,["travel_time"]]=data.loc[:,["travel_time"]].values*data.loc[:,["n_vehicles"]].values #travel time*n_vehicles to get the right value after the groupby function
data = data.groupby(['from_stop_I', 'to_stop_I', "route_type", "route_I"]).agg({"travel_time": "sum",
                                                                    "n_vehicles": "sum"}).reset_index() #merge mostly duplicate rows (i.e. same edges in the network) 
data.loc[:,["travel_time"]]=data.loc[:,["travel_time"]].values/data.loc[:,["n_vehicles"]].values  #travel time/n_vehicles to derive the right values out of the groupby function                                                          
data['route_I_counts'] = data[["route_I", 'n_vehicles']].astype(str).apply(lambda x: ':'.join(x), axis=1) #create the route_I_counts column  
data = data.drop(columns = ["route_I"]) #drop route_I
data.loc[:,["travel_time"]] = data.loc[:,["travel_time"]].values*data.loc[:,["n_vehicles"]].values #travel time*n_vehicles to get the right value after the groupby function
data = data.groupby(['from_stop_I', 'to_stop_I', "route_type"]).agg({"travel_time": "sum",
                                                                    "n_vehicles": "sum",
                                                                    'route_I_counts': ','.join}).reset_index()
data.loc[:,["travel_time"]]=data.loc[:,["travel_time"]].values/data.loc[:,["n_vehicles"]].values #travel time/n_vehicles to derive the right values out of the groupby function


for row in data.itertuples(): #adding the "d" values from the combined edge list to the newly developed temporal event list 
  for row1 in combined_L.itertuples():
    if row[1] == row1[1] and row[2] == row1[2]:
      data.at[row.Index, "d"] = row1[3]
      
data = data.rename(columns = {"travel_time": "duration_avg"}) #rename column
data = data[["from_stop_I", "to_stop_I", "d", "duration_avg", "n_vehicles", "route_I_counts", "route_type"]] #Columns back in the right order
data.to_csv(DATA_PATH + "/rome/combined_L_11am_12am.csv", index = False)

# **LOADING DATA**

In [0]:
##Monday 8am-9am data
combined_L_89 = pd.read_csv(DATA_PATH + "/rome/combined_L_8am_9am.csv") #combined data, as processed above
tram_L_89 = combined_L_89[combined_L_89["route_type"]==0] #tram data, derived from the combined data
tram_L_89 = tram_L_89.drop(columns = ["route_type"])
rail_L_89 = combined_L_89[combined_L_89["route_type"]==2] #rail data, derived from the combined data
rail_L_89 = rail_L_89.drop(columns = ["route_type"])
subway_L_89 = combined_L_89[combined_L_89["route_type"]==1] #subway data, derived from the combined data
subway_L_89 = subway_L_89.drop(columns = ["route_type"])
bus_L_89 = combined_L_89[combined_L_89["route_type"]==3] #bus data, derived from the combined data
bus_L_89 = bus_L_89.drop(columns = ["route_type"])

##Monday 8am-9am data
combined_L_1112 = pd.read_csv(DATA_PATH + "/rome/combined_L_11am_12am.csv") #combined data, as processed above
tram_L_1112 = combined_L_1112[combined_L_1112["route_type"]==0] #tram data, derived from the combined data
tram_L_1112 = tram_L_1112.drop(columns = ["route_type"])
rail_L_1112 = combined_L_1112[combined_L_1112["route_type"]==2] #rail data, derived from the combined data
rail_L_1112 = rail_L_1112.drop(columns = ["route_type"])
subway_L_1112 = combined_L_1112[combined_L_1112["route_type"]==1] #subway data, derived from the combined data
subway_L_1112 = subway_L_1112.drop(columns = ["route_type"])
bus_L_1112 = combined_L_1112[combined_L_1112["route_type"]==3] #bus data, derived from the combined data
bus_L_1112 = bus_L_1112.drop(columns = ["route_type"])

##Walking distance
walking_distance = pd.read_csv(DATA_PATH + '/rome/network_walk.csv', sep = ";") #dataset comprising walking distances

##Node data
network_nodes = pd.read_csv(DATA_PATH + "/rome/network_nodes.csv", sep = ";") #node list, comprising coordinates and name of stops

##GeoJSON_files
routes_geojson = gpd.read_file(DATA_PATH + '/rome/routes.geojson') #Public transport routes in GeoJSON format.
sections_geojson = gpd.read_file(DATA_PATH + '/rome/sections.geojson') #Each stop-to-stop section in GeoJSON format.
stops_geojson = gpd.read_file(DATA_PATH + '/rome/stops.geojson') #Information on the nodes in GeoJSON format.

# **Graph Networkx**


# **L-Space Graph**


In [0]:
#The code below creates the L_space graph for 8am-9am interval
combined_L_89_graph = nx.from_pandas_edgelist(combined_L_89, source = "from_stop_I", target = "to_stop_I", edge_attr = ("duration_avg", "n_vehicles", "route_I_counts", "d", "route_type"))

In [0]:
#The code below creates the L_space graphs for 11am-12am interval
combined_L_1112_graph = nx.from_pandas_edgelist(combined_L_1112, source = "from_stop_I", target = "to_stop_I", edge_attr = ("duration_avg", "n_vehicles", "route_I_counts", "d", "route_type"))

# **PREPROCESSING**

## Algorithm to create the supernode network representation

In [0]:
def sub_graph_detector(tram_graph, rail_graph , subway_graph, bus_graph, combined_graph):
  """
  This code has been used to detect disconnected subnetworks, as a first step to
  identify temporal or spatial artefacts in the dataset as provided by Kujala et
  al. (2018) or as a result of preprocessing.
  """
  sub_dict = {"tram":nx.connected_components(tram_graph), "rail": nx.connected_components(rail_graph),
              "subway": nx.connected_components(subway_graph), "bus": nx.connected_components(bus_graph),
              "combined": nx.connected_components(combined_graph)}

  for k,v in sub_dict.items():
    count = 0
    print()
    print(k)
    for i, sg in enumerate(v):   
      print("Subgraph {} has {} nodes".format(i+1, len(sg)))
      count+= len(sg)
    print("Total number of nodes:", count)
    print("")
      
sub_graph_detector(tram_L_89_graph, rail_L_89_graph , subway_L_89_graph, bus_L_89_graph, combined_L_89_graph)
print("=============================================")
sub_graph_detector(tram_L_1112_graph, rail_L_1112_graph , subway_L_1112_graph, bus_L_1112_graph, combined_L_1112_graph)

In [0]:
def remove_cut_off_nodes(G_L):
  """ 
  After a check in the geojson files and a validation in Google maps,
  there appear to be three nodes as subgraph in the bus and combined data that 
  have been cut of due to the 20km radius. These can be removed with this 
  function. In addition: in the rail (and thus also combined) data of the
  interval 11am-12am, there are three stations that are disconnected both from
  the route they are part of as well as from other stops in walking distance,
  due to temporal artefacts.  These have been removed too.
  See table 4 in the paper for further explanation.
  """
  node_list = []
  for sub_graph in nx.connected_components(G_L):
    if len(sub_graph) <5:
      for n in sub_graph:
        node_list.append(n)
  for node in node_list:
    #G_P.remove_node(node)
    G_L.remove_node(node)
  print("removed nodes", node_list)

remove_cut_off_nodes(combined_L_89_graph)
remove_cut_off_nodes(combined_L_1112_graph)

In [0]:
def identify_near_nodes(walk_distance):
  """
  This function identifies the nodes that are at a distance of max the
  determined merge_radius (and thus are going to be transformed into 
  supernodes)
  """
  merge_radius = 100
  near_nodes = []
  for index, row in walk_distance.iterrows():
    if row["d_walk"] < merge_radius:
      near_nodes.append((row["from_stop_I"], row["to_stop_I"]))
    else:
      continue
  return near_nodes

near_nodes = identify_near_nodes(walking_distance)
print(near_nodes)

In [0]:
def create_supernodes(G, near_nodes):
  """
  This code creates the supernode network structure
  """ 
  for u,v in near_nodes:
    if u in nx.nodes(G) and v in nx.nodes(G):
      G = nx.contracted_nodes(G, u, v, self_loops=False)
  return G

#create new graphs with supernodes and save them into variables
temp_combined_L_graph_sn1112 = create_supernodes(combined_L_1112_graph, near_nodes)
temp_combined_L_graph_sn1112 = create_supernodes(combined_L_1112_graph, near_nodes)

Saving the supernode graphs, so that we don't have to run the code every time
again

In [0]:
#saving the 8am-9am L-space graphs with supernodes into a pickle file
nx.write_gpickle(temp_combined_L_graph_sn, DATA_PATH + "/rome/combined89_L_graph_sn")

In [0]:
#saving the 11am-12am L-space graphs with supernodes into a pickle file
nx.write_gpickle(temp_combined_L_graph_sn1112, DATA_PATH + "/rome/combined1112_L_graph_sn")

In [0]:
#uploading the supernode Lspace graphs from the pickle files written in 
#the code above1112
combined89_L_graph_sn = nx.read_gpickle(DATA_PATH + "/rome/combined89_L_graph_sn")
combined1112_L_graph_sn = nx.read_gpickle(DATA_PATH + "/rome/combined1112_L_graph_sn")

In [0]:
#Uploading graphs into edgelists
combined89_L_list_sn = nx.to_pandas_edgelist(combined89_L_graph_sn)
combined1112_L_list_sn = nx.to_pandas_edgelist(combined1112_L_graph_sn)

## Convert L-space to P-space

Combined 8am-9am & 11am-12am

This code creates the unweighted P-space edgelist, see example Table 5 in paper

In [0]:
#Code below transforms L-space data into P-space data for combined data
#Only feasible for combined data
def drop_unnecessary_L_data_C89(L_space):
  df = L_space.copy()
  df[["route_I_counts_1", "route_I_counts_2", "route_I_counts_3", "route_I_counts_4",
      "route_I_counts_5", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"]] = df.route_I_counts.str.split(",", expand = True) #putting the different route counts into separate columns
  df = df.drop(columns = ["route_I_counts", "d", "duration_avg", "n_vehicles"]) #drop all unnecessary columns
  df['route_I_counts_1'] = df['route_I_counts_1'].str.split(":", expand = True) #remove count for routes --> In case I need count later, make a new column for count: df['route_I_1', "route_I_1_count"] =\
  df['route_I_counts_2'] = df['route_I_counts_2'].str.split(":", expand = True) #remove count for routes
  df['route_I_counts_3'] = df['route_I_counts_3'].str.split(":", expand = True) #remove count for routes
  df['route_I_counts_4'] = df['route_I_counts_4'].str.split(":", expand = True) #remove count for routes
  df['route_I_counts_5'] = df['route_I_counts_5'].str.split(":", expand = True) #remove count for routes
  df['route_I_counts_6'] = df['route_I_counts_6'].str.split(":", expand = True) #remove count for routes
  df['route_I_counts_7'] = df['route_I_counts_7'].str.split(":", expand = True) #remove count for routes
  df['route_I_counts_8'] = df['route_I_counts_8'].str.split(":", expand = True) #remove count for routes
  df['route_I_counts_9'] = df['route_I_counts_9'].str.split(":", expand = True) #remove count for routes
  df['route_I_counts_10'] = df['route_I_counts_10'].str.split(":", expand = True) #remove count for routes
  df['route_I_counts_11'] = df['route_I_counts_11'].str.split(":", expand = True) #remove count for routes
  df.fillna(value = int(0), inplace = True)
  df = df.astype(str).astype(int)
  return df

def sep_routes_C89(L_space):
  routes = drop_unnecessary_L_data_C89(L_space)
  """
  Cleans routes dataframe. Returns a dataframe with stops (u,v) and corresponding route. 
  Always check maximum amount of separate routes on one line in mode dataset,
  adapt the amount of created "route_n" variables to that.
  """
  df_new = pd.DataFrame(columns= ["from_stop_I", "to_stop_I", "route_type", "route_I"])
  route_1 = routes.loc[routes["route_I_counts_1"] != 0 ]
  route_1 = route_1.drop(columns = ["route_I_counts_2", "route_I_counts_3", "route_I_counts_4",
      "route_I_counts_5", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_1.columns = ["from_stop_I", "to_stop_I", "route_type", "route_I"]
  route_2 = routes.loc[routes["route_I_counts_2"] != 0 ]
  route_2 = route_2.drop(columns = ["route_I_counts_1", "route_I_counts_3","route_I_counts_4",
      "route_I_counts_5", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_2.columns = ["from_stop_I", "to_stop_I", "route_type", "route_I"]
  route_3 = routes.loc[routes["route_I_counts_3"] != 0 ]
  route_3 = route_3.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_4",
      "route_I_counts_5", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_3.columns = ["from_stop_I", "to_stop_I", "route_type", "route_I"]
  route_4 = routes.loc[routes["route_I_counts_4"] != 0 ]
  route_4 = route_4.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_5", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_4.columns = ["from_stop_I", "to_stop_I", "route_type", "route_I"]
  route_5 = routes.loc[routes["route_I_counts_5"] != 0 ]
  route_5 = route_5.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_5.columns = ["from_stop_I", "to_stop_I", "route_type", "route_I"]
  route_6 = routes.loc[routes["route_I_counts_6"] != 0 ]
  route_6 = route_6.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_6.columns = ["from_stop_I", "to_stop_I", "route_type", "route_I"]
  route_7 = routes.loc[routes["route_I_counts_7"] != 0 ]
  route_7 = route_7.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_6", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_7.columns = ["from_stop_I", "to_stop_I", "route_type", "route_I"]
  route_8 = routes.loc[routes["route_I_counts_8"] != 0 ]
  route_8 = route_8.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_6", "route_I_counts_7",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_8.columns = ["from_stop_I", "to_stop_I", "route_type", "route_I"]
  route_9 = routes.loc[routes["route_I_counts_9"] != 0 ]
  route_9 = route_9.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_6", "route_I_counts_7",
      "route_I_counts_8", "route_I_counts_10", "route_I_counts_11"])
  route_9.columns = ["from_stop_I", "to_stop_I", "route_type", "route_I"]
  route_10 = routes.loc[routes["route_I_counts_10"] != 0 ]
  route_10 = route_10.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_6", "route_I_counts_7",
      "route_I_counts_8", "route_I_counts_9", "route_I_counts_11"])
  route_10.columns = ["from_stop_I", "to_stop_I", "route_type", "route_I"]
  route_11 = routes.loc[routes["route_I_counts_11"] != 0 ]
  route_11 = route_11.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_6", "route_I_counts_7",
      "route_I_counts_8", "route_I_counts_9", "route_I_counts_10",])
  route_11.columns = ["from_stop_I", "to_stop_I", "route_type", "route_I"]

  pdList = [route_1, route_2, route_3, route_4, route_5, route_6, route_7,
            route_8, route_9, route_10, route_11]
  df_new = pd.concat(pdList)
  df_new = df_new.sort_index()
  df_new["route_I"] = df_new["route_I"].astype(str).astype("int64")
  df_new = df_new.sort_values(by = ["route_I"])
  return df_new

def create_dict_C89(L_space):
  """
  This transforms the dataset created above into a dictionary with the routes 
  as keys and the stops on that route as values. For example, route A : [1, 2, 3, 4....]
  In the combined dataset the keys consist of two values in a tuple: (route, mode of transport)
  """
  L_space = sep_routes_C89(L_space)
  L_space = L_space.groupby(["route_I", "route_type"])
  dict1 =  L_space["from_stop_I"].unique().to_dict()
  dict2 =  L_space["to_stop_I"].unique().to_dict()
  ds = [dict1, dict2]
  d = {}
  for k in dict1.keys():
    d[k] = np.concatenate(list(d[k] for d in ds))
  for k,v in d.items():
    d[k] = np.unique(v)
  return d

def P_space_C89(L_space):
  """
  Converts the dictionary created above into a dataframe consisting of P_space data
  """
  L_space_dict = create_dict_C89(L_space)
  df_list = []
  for k,v in L_space_dict.items():
    for values in v:
      for values2 in v:
        if values != values2:
          df_list.append({"from_stop_I": values, "to_stop_I": values2, "route_I": k[0], "route_type": k[1]})
        else:
          continue
  result = pd.DataFrame(df_list)
  return result

In [0]:
#convert the supernode L_space edgelist to P-space edgelists with the code
#written aboven 
combined89_P_list_sn = P_space_C89(combined89_L_list_sn)
combined1112_P_list_sn = P_space_C89(combined1112_L_list_sn)

# **MODEL**

## Compute in vehicle travel time

In [0]:
def add_travel_time(mode_P_edgelist, mode_L_graph ):
    '''
    calculates travel time between all connected stops in P space, used for
        supernode data:
        mode_P_edgelist = P_space pandas_edgelist with origin and destination 
        node(stop) and route(line) number. In addition, combined data consists 
        of route mode column. 
        mode_L_graph = networkx graph of the L-space of the same mode as 
        mode_P_edgelist.
    returns:
        Pandas dataframe (edgelist) consisting of columns origin node and 
        destination node, route(line) number, route mode when combined data, and
        travel time.
    '''
    mode_shortest_path = dict(nx.all_pairs_dijkstra_path_length(mode_L_graph, weight = "duration_avg"))
    for row in mode_P_edgelist.itertuples():
      mode_P_edgelist.at[row.Index, "travel_time"] = mode_shortest_path[row[1]][row[2]]
    return mode_P_edgelist



combined89_P_list_sn = add_travel_time(combined89_P_list_sn, combined89_L_graph_sn) #be aware: both the P-edgelist and supernode graph need to be up to date
combined1112_P_list_sn = add_travel_time(combined1112_P_list_sn, combined1112_L_graph_sn) #be aware: both the P-edgelist and supernode graph need to be up to date


In [0]:
## Load to new P-space data into csv files, so that the code above doesnt have to be
## runned every time again
combined89_P_list_sn.to_csv(DATA_PATH + "/rome/combined89_P_list_sn_tt.csv", index = False)
combined1112_P_list_sn.to_csv(DATA_PATH + "/rome/combined1112_P_list_sn_tt.csv", index = False)

In [0]:
##Load the P-space data from the csv files into variables
combined89_P_list_sn = pd.read_csv(DATA_PATH + "/rome/combined89_P_list_sn_tt.csv")
combined1112_P_list_sn = pd.read_csv(DATA_PATH + "/rome/combined1112_P_list_sn_tt.csv")

## Compute waiting time


Combined

In [0]:
"""
The code below creates a new column in the P-space edge list: waiting time.
The waiting time is calculated as half of the time between the joint service
frequency of an edge.
"""

def sep_route_counts_C(L_space):
  """
  This function puts the joint route count per edge in L-space into separate columns
  """ 
  df = L_space.copy()
  df[["route_I_counts_1", "route_I_counts_2", "route_I_counts_3", "route_I_counts_4",
      "route_I_counts_5", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"]] = df.route_I_counts.str.split(",", expand = True)
  df.fillna(value = int(0), inplace = True)
  return df

def sep_route_C(L_space):
  """
  This function transforms the dataset created above.
  Basically, what happens is that every single route on an edge gets a distinct
  row. This codes creates an easily readable dataFrame in L-space, necessary to
  count the frequency per edge (see next code).
  """  
  routes = sep_route_counts_C(L_space)
  df_new = pd.DataFrame(columns= ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"])
  route_1 = routes.loc[routes["route_I_counts_1"] != 0 ]
  route_1 = route_1.drop(columns = ["route_I_counts_2", "route_I_counts_3", "route_I_counts_4",
      "route_I_counts_5", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_1.columns = ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"]
  route_2 = routes.loc[routes["route_I_counts_2"] != 0 ]
  route_2 = route_2.drop(columns = ["route_I_counts_1", "route_I_counts_3","route_I_counts_4",
      "route_I_counts_5", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_2.columns = ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"]
  route_3 = routes.loc[routes["route_I_counts_3"] != 0 ]
  route_3 = route_3.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_4",
      "route_I_counts_5", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_3.columns = ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"]
  route_4 = routes.loc[routes["route_I_counts_4"] != 0 ]
  route_4 = route_4.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_5", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_4.columns = ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"]
  route_5 = routes.loc[routes["route_I_counts_5"] != 0 ]
  route_5 = route_5.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_6", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_5.columns = ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"]
  route_6 = routes.loc[routes["route_I_counts_6"] != 0 ]
  route_6 = route_6.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_7", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_6.columns = ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"]
  route_7 = routes.loc[routes["route_I_counts_7"] != 0 ]
  route_7 = route_7.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_6", "route_I_counts_8",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_7.columns = ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"]
  route_8 = routes.loc[routes["route_I_counts_8"] != 0 ]
  route_8 = route_8.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_6", "route_I_counts_7",
      "route_I_counts_9", "route_I_counts_10", "route_I_counts_11"])
  route_8.columns = ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"]
  route_9 = routes.loc[routes["route_I_counts_9"] != 0 ]
  route_9 = route_9.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_6", "route_I_counts_7",
      "route_I_counts_8", "route_I_counts_10", "route_I_counts_11"])
  route_9.columns = ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"]
  route_10 = routes.loc[routes["route_I_counts_10"] != 0 ]
  route_10 = route_10.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_6", "route_I_counts_7",
      "route_I_counts_8", "route_I_counts_9", "route_I_counts_11"])
  route_10.columns = ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"]
  route_11 = routes.loc[routes["route_I_counts_11"] != 0 ]
  route_11 = route_11.drop(columns = ["route_I_counts_1", "route_I_counts_2","route_I_counts_3",
      "route_I_counts_4", "route_I_counts_5", "route_I_counts_6", "route_I_counts_7",
      "route_I_counts_8", "route_I_counts_9", "route_I_counts_10"])
  route_11.columns = ["from_stop_I", "to_stop_I", "d", "duration_avg", "route_type", "route_I_counts", "n_vehicles", "route_I"]
  pdList = [route_1, route_2, route_3, route_4, route_5, route_6, route_7,
            route_8, route_9, route_10, route_11]
  df_new = pd.concat(pdList)
  df_new = df_new.sort_index()
  return df_new

def route_count_C(L_space):
  """
  This function creates a dataframe consisting of the two adjacent stops of a node,
  its route number and the frequency of a particular route on that edge.
  In other words, it creates a quick overview of the frequency per edge, per route.
  """ 
  L_space = sep_route_C(L_space) #Apply the codes written above to the L-space data
  L_space = L_space.drop(columns = ["route_I_counts", "d", "duration_avg", "n_vehicles"]) #drop all unnecessary columns
  L_space[["route_I", "n_vehicles"]] = L_space.route_I.str.split(":", expand = True) #derive the right route_I and n_vehicles from route_I_counts
  L_space["n_vehicles"] = L_space["n_vehicles"].astype(str).astype("int64") #n_vehicles as integer
  return L_space

def add_waiting_time_C(L_space, P_space):
  """
  By selecting the rouded-up average of the edges belonging to one route, this
  function is able to calculate the frequency per route. After calculating the
  frequency this function calculates the waiting time (see section 3 in methods
  for more explanation) and adds the waiting time in the P-space DataFrame
  edgelist.
  """
  P_space = P_space.copy()
  L_space = route_count_C(L_space) #Apply the codes written above to the L-space data
  frequency_dict =  np.ceil(L_space.groupby(["route_I"]).n_vehicles.mean()).to_dict() #retrieve the frequency of routes and place them into a dict
  #This is done by calculating the ceiled mean of the number of vehicles that drive between nodes, separated by route

  for row in P_space.itertuples():
    for k,v in frequency_dict.items():
      if row[3] == int(k): #if route_i is similar to route_i in dictionary keys
        P_space.at[row.Index, "frequency"] = int(v) #Creates the column frequency in P-space which consists of the frequency per route as calculated in frequency_dict

  P_space['route_I'] = P_space['route_I'].astype(str)
  P_space = P_space.groupby(['from_stop_I', 'to_stop_I', "route_type", "travel_time"]).agg({"frequency": "sum",
                                                                    'route_I': ','.join}).reset_index() 
  #sums the frequency per edge in P-space and transfers multi edge into single edge with the join function.
  #In other words: if two adjacent nodes are connected by more than one route of the same mode of transport, the different routes are merged

  P_space["waiting_time"] = (60*60)/(2*P_space["frequency"]) #create new column waiting time: 3600 seconds (timespan used) divided by 2 times the headway
  P_space = P_space[["from_stop_I", "to_stop_I", "route_type", "route_I", "frequency", "travel_time", "waiting_time"]] #rearrange column order

  return P_space

In [0]:
#Apply code
combined89_P_sn = add_waiting_time_C(combined89_L_list_sn, combined89_P_list_sn )
combined1112_P_sn = add_waiting_time_C(combined1112_L_list_sn, combined1112_P_list_sn )

In [0]:
#The code below saves the new P-space data set (including travel time and waiting
#as separate columns), in order to prevent that we have to run the code everytime again
combined89_P_sn.to_csv(DATA_PATH + "/rome/combined89_P_sn_ttwt.csv", index = False)
combined1112_P_sn.to_csv(DATA_PATH + "/rome/combined1112_P_sn_ttwt.csv", index = False)

## Compute weight (consisting of waiting time and travel time)


In [0]:
#loading data saved above
combined89_P_sn = pd.read_csv(DATA_PATH + "/rome/combined89_P_sn_ttwt.csv")
combined1112_P_sn = pd.read_csv(DATA_PATH + "/rome/combined1112_P_sn_ttwt.csv")

In [0]:
def compute_weight(P_space):
  """
  Adds the waiting time to the in-vehicle travel time and creates a new column
    in the dataset comprising the total travel time which can be used as 
    weight for the edges in the network representation.
  returns:
    Pandas dataframe (edgelist) with extra column "weight"
  """
  P_space["weight"] = P_space["travel_time"] + P_space["waiting_time"]
  return P_space

In [0]:
#applying compute_weight function
combined89_P_sn = compute_weight(combined89_P_sn)
combined1112_P_sn = compute_weight(combined1112_P_sn)

In [0]:
def to_multigraph_data(P_space):
  """
  Important note: this step is only needed for a multimodal dataset (for single-modal datasets: nx.Graph will do the same)
  Seen the fact that there are duplicate edges in the data (edges wit same source and target nodes, but different modes of transport)
    We need a multigraph (data set) that makes a distinction in the two different modes of transport between two adjacent stops (in case they exist).
    This code converts the P_space data set with all edge attributes into a database that can directly be implemented as a multigraph
    in the shortest path analyzer needed to calculate the generalised travel costs per link in P_space.
    For the database, this means, firstly, that we get rid of directed edges and we convert them into undirected edges. This is a necessary step to take
    since we make use of a supernode graph structure (nodes on <=100 m walking distance have been contracted (e.g. nodes on opposite side of the road)). 
    Secondly, since we cannot use a Graph (we need a Multigraph for edges between the same pair of nodes, but with a different mode of transport),
    we have to remove duplicate edges manually (this are edges that were directed first, but double in an undirected multigraph) 
  returns:
    Pandas dataframe (edgelist) ready to use for the shortest path analyer that calculates the travel generalised travel costs per link in P_space
  """
  P_space = P_space.copy()
  P_space = nx.from_pandas_edgelist(P_space, source = "from_stop_I", target = "to_stop_I", 
            edge_attr = ["route_I", "route_type", "frequency", "travel_time", "waiting_time", "weight"], create_using = nx.MultiGraph())
  P_space = nx.to_pandas_edgelist(P_space)
  P_space = P_space.drop_duplicates()
  P_space = P_space.reset_index(drop = True)
  return P_space

In [0]:
#applying to_multigraph_data function
combined89_P_sn = to_multigraph_data(combined89_P_sn)
combined1112_P_sn = to_multigraph_data(combined1112_P_sn)

In [0]:
#Save dataset
combined89_P_sn.to_csv(DATA_PATH + "/rome/combined89_P_sn_weight.csv", index = False)
combined1112_P_sn.to_csv(DATA_PATH + "/rome/combined1112_P_sn_weight.csv", index = False)

# Convert P-edgelist to graph



In [0]:
#Load data
combined89_P_sn = pd.read_csv(DATA_PATH + "/rome/combined89_P_sn_weight.csv")
combined1112_P_sn = pd.read_csv(DATA_PATH + "/rome/combined1112_P_sn_weight.csv")

In [0]:
#Convert the edgelist into a graph ready to use for the shortest path analyzer to generate generalised travel costs between node pairs
rome_89_P_graph = nx.from_pandas_edgelist(combined89_P_sn, source = "source", target = "target", 
            edge_attr = ["route_I", "route_type", "frequency", "travel_time", "waiting_time", "weight"], create_using = nx.MultiGraph())


In [0]:
#Convert the edgelist into a graph ready to use for the shortest path analyzer to generate generalised travel costs between node pairs
rome_1112_P_graph = nx.from_pandas_edgelist(combined1112_P_sn, source = "source", target = "target", 
            edge_attr = ["route_I", "route_type", "frequency", "travel_time", "waiting_time", "weight"], create_using = nx.MultiGraph())

In [0]:
#Add coordinates and node names as node attributes to the graph
network_nodes['coord'] = list(zip(network_nodes.lon, network_nodes.lat))
pos = network_nodes.set_index("stop_I")["coord"].to_dict()

def add_coordinates(network_nodes, G):
  """
    network_nodes = csv file with network node coordinates
    G = networkx (multi)graph
  This function adds the coordinates to the nodes in the selected graph
  returns: networkx (multi)graph 
  """
  network_nodes['coord'] = list(zip(network_nodes.lon, network_nodes.lat))
  pos = network_nodes.set_index("stop_I")["coord"].to_dict()
  nx.set_node_attributes(G, pos, name = "pos")
  return G

def add_name(network_nodes, G):
  """
    network_nodes = csv file with network node coordinates
    G = networkx (multi)graph
  This function adds the name of the stops to the nodes in the selected graph
  returns: networkx (multi)graph 
  """
  name = network_nodes.set_index("stop_I")["name"].to_dict()
  nx.set_node_attributes(G, name, "name" )
  return G

rome_89_P_graph = add_coordinates(network_nodes, rome_89_P_graph)
rome_89_P_graph = add_name(network_nodes, rome_89_P_graph)
rome_89_P_graph.nodes(data= True) #should reflect pos and name per node 
rome_1112_P_graph = add_coordinates(network_nodes, rome_1112_P_graph)
rome_1112_P_graph = add_name(network_nodes, rome_1112_P_graph)
rome_1112_P_graph.nodes(data= True) #should reflect pos and name per node 

# Select giant connected component


Below we select the giant connected component
In Rome there is a small part of the network (26 nodes out of 5419 in total) that is not connected, for the accessibility analysis these are not included.

In [0]:
def connected_component_subgraphs(G):
  """
  This function selects the connected components in a graph
  """
  for c in nx.connected_components(G):
      yield G.subgraph(c)

def subgraph_check(G):
  """
  This function puts the nodes and edges of the subgraphs into dictionaries.
  Furthermore, this function prints the number of nodes and edges per subgraph
  Input: nx.Graph
  Returns: two dictionaries. First consists of subgraphs and corresponding nodes.
  Second consists of subgraphs and corresponding edges.
  """
  count = 1
  subgraph_nodes_dict = dict()
  subgraph_edges_dict = dict()
  for g in connected_component_subgraphs(G):
    print("number of nodes {}:".format(count), g.number_of_nodes())
    nodes_list = list(g.nodes)
    subgraph_nodes_dict[count] = nodes_list
    print("number of edges {}:".format(count), g.number_of_edges())
    edges_list = list(g.nodes)
    subgraph_edges_dict[count] = edges_list = list(g.nodes)
    count += 1
  return subgraph_nodes_dict, subgraph_edges_dict


In [0]:
nodes_subgraph_rome89, edges_subgraph_rome89 = subgraph_check(rome_89_P_graph)

In [0]:
nodes_subgraph_rome1112, edges_subgraph_rome1112 = subgraph_check(rome_1112_P_graph)

In [0]:
## This code puts the subgraphs into separate variables, be aware of the fact that the right amount of 
## variables has been used before the "="

rome_89_largest_cc, rome_89_smaller_cc_1 = [rome_89_P_graph.subgraph(c).copy() for c in nx.connected_components(rome_89_P_graph)]
##The smaller connected component is a busroute not connected to the rest of the network, see subgraph_detection function for more information

In [0]:
# This code puts the subgraphs into separate variables, be aware of the fact that the right amount of 
# variables has been used before the "="

rome_1112_largest_cc, rome_1112_smaller_cc_1, rome_1112_smaller_cc_2 = [rome_1112_P_graph.subgraph(c).copy() for c in nx.connected_components(rome_1112_P_graph)]
#The smaller connected components are: (1) rome_1112_smaller_cc_1, a railroute (331 & 335) not connected to the rest of the network
#(2) rome_89_smaller_cc_1, a busroute not connected to the rest of the network

In [0]:
#save GCC
nx.write_gpickle(rome_1112_largest_cc, DATA_PATH + "/rome/rome1112_P_GCC")
nx.write_gpickle(rome_89_largest_cc, DATA_PATH + "/rome/rome89_P_GCC")

In [0]:
#load GCC
rome_89_largest_cc = nx.read_gpickle(DATA_PATH + "/rome/rome89_P_GCC")
rome_1112_largest_cc = nx.read_gpickle(DATA_PATH + "/rome/rome1112_P_GCC")

# Compute generalised travel costs for every node-pair in the network


In [0]:
def generate_GTC_matrix(G):
  """
  input: Graph with edge attribute "weight"
  returns: three dataframe matrices. (1) transfer penalty time
  (2) travel time (consisting of in vehicle travel time and waiting time)
  (3) Sum of both: generalized travel costs
  """
  dijkstra_path_length = dict(nx.all_pairs_dijkstra(G, weight = "weight"))
  length = dict()
  path = dict()
  for key, values in dijkstra_path_length.items():
    length[key] = values[0]
    path[key] = values[1]

  """
  The code below calculates the number of transfers made from all to all nodes
  with all_pairs_dijkstra_shortest_path

  """
  df_path = pd.DataFrame(path)
  df_path = df_path.astype('str')
  df_path = df_path.replace('\[','', regex = True, inplace=False)
  df_path = df_path.replace(']','', regex = True, inplace = False)
  count_transfer = []
  for index, row in df_path.iterrows():
    A = row.str.count(',')
    count_transfer.append(A)
  df_transfer = pd.DataFrame(count_transfer)

  """
  The code below, transfers are being penalized with a 300 seconds (5 minutes) penalty
  input: Graph with edge attribute "weight"
  returns: dataframe matrix with count of transfers
  """
  
  transfer_penalty = lambda x: 300*(x-1)
  df_transfer = transfer_penalty(df_transfer)
  df_transfer[df_transfer == (-300)] = 0 
  df_transfer = df_transfer.sort_index(axis=0)
  df_transfer = df_transfer.sort_index(axis=1)
  
  """
  Calculates the travel time consisting of in-vehicle travel time and waiting time
  with all_pairs_dijkstra_shortest_path
  input: Graph with edge attribute "weight"
  returns: dataframe matrix with travel time
  """
  travel_time = length
  df_travel = pd.DataFrame(length)             
  df_travel = df_travel.sort_index(axis=0)
  df_travel = df_travel.sort_index(axis=1)


  """
  The code below calculates the Generalised Travel Costs consisting of in-vehicle travel time,
  waiting time and transfer penalty with all_pairs_dijkstra_shortest_path
  In line with method developed by Luo et al. 2019 (see thesis report for complete
  reference)
  input: Graph with edge attribute "weight"
  returns: dataframe matrix with GTC
  """ 

  df_GTC = df_travel.add(df_transfer, fill_value=0)
  df_GTC = df_GTC.sort_index(axis=0)
  df_GTC = df_GTC.sort_index(axis=1)


  return df_transfer, df_travel, df_GTC

In [0]:
#apply code and save matrices
rome_1112_transfer, rome_1112_travel, rome_1112_gtc = generate_GTC_matrix(rome_1112_largest_cc)
rome_1112_gtc.to_csv(DATA_PATH + "/rome1112_final_gtc.csv", index = True)
rome_1112_transfer.to_csv(DATA_PATH + "/rome1112_final_transfer.csv", index = True)
rome_1112_travel.to_csv(DATA_PATH + "/rome1112_final_travel.csv", index = True)

In [0]:
#apply code and save matrices
rome_89_transfer, rome_89_travel, rome_89_gtc = generate_GTC_matrix(rome_89_largest_cc)
rome_89_gtc.to_csv(DATA_PATH + "/rome89_final_gtc.csv", index = True)
rome_89_transfer.to_csv(DATA_PATH + "/rome89_final_transfer.csv", index = True)
rome_89_travel.to_csv(DATA_PATH + "/rome89_final_travel.csv", index = True)

In [0]:
#Load GTC matrix 8-9AM
rome_89_gtc = pd.read_csv(DATA_PATH + "/rome89_final_gtc.csv", sep=",")
rome_89_gtc = rome_89_gtc.set_index("Unnamed: 0")
rome_89_gtc.index.name = None


In [0]:
#Load GTC matrix 8-9AM
rome_1112_gtc = pd.read_csv(DATA_PATH + "/rome1112_final_gtc.csv", sep=",")
rome_1112_gtc = rome_1112_gtc.set_index("Unnamed: 0")
rome_1112_gtc.index.name = None
rome_1112_gtc

In [0]:
#calculate Travel Impedance per node in 11-12AM data
rome_1112_travel_impedance = rome_1112_gtc.sum()/5375  #sum/(sum-1)
rome_1112_travel_impedance = pd.DataFrame(rome_1112_travel_impedance)

In [0]:
#Calculate Travel Impedance per node in 8-9AM data
rome_89_travel_impedance = rome_89_gtc.sum()/5397 #sum/(sum-1)
rome_89_travel_impedance = pd.DataFrame(rome_89_travel_impedance)

# Compute in-vehicle travel time for every node-pair in the P-space network representation (needed for violin plot section 4.2)

The code below calculates the in-vehicle travel time for every node pair, in exactly the same way as how the GTC has been calculated

In [0]:
def generate_IVTT_matrix(G):
  """
  input: Graph with edge attribute "weight"
  returns: pd.DataFrame matrix with total in vehicle travel time as gtc-component
  """
  length = dict(nx.all_pairs_dijkstra_path_length(G, weight = "travel_time"))

  """
  Calculates the travel time consisting of in-vehicle travel time and waiting time
  with all_pairs_dijkstra_shortest_path
  input: Graph with edge attribute "weight"
  returns: dataframe matrix with travel time
  """
  df_travel = pd.DataFrame(length)             
  df_travel = df_travel.sort_index(axis=0)
  df_travel = df_travel.sort_index(axis=1)

  return df_travel

In [0]:
rome89_IVTT = generate_IVTT_matrix(rome_89_largest_cc)

In [0]:
rome112_IVTT = generate_IVTT_matrix(rome_1112_largest_cc)

In [0]:
rome89_IVTT.to_csv(DATA_PATH + "/rome/rome89_IVTT.csv", index = True)

In [0]:
rome112_IVTT.to_csv(DATA_PATH + "/rome/rome1112_IVTT.csv", index = True)

In [0]:
rome89_IVTT = pd.read_csv(DATA_PATH + "/rome/rome89_IVTT.csv", sep=",")
rome89_IVTT = rome89_IVTT.set_index("Unnamed: 0")
rome1112_IVTT = pd.read_csv(DATA_PATH + "/rome/rome1112_IVTT.csv", sep=",")
rome1112_IVTT = rome1112_IVTT.set_index("Unnamed: 0")

# Compute waiting + transfer time for every node pair in the P-space network representation (needed for violin plot section 4.2)

The code below calculates the waiting time + transfer penalty for every node pair, in exactly the same way as how the GTC has been calculated

In [0]:
def generate_WT_matrix(G):
  """
  input: Graph with edge attribute "waiting_time"
  returns: pd.DataFrame matrix with total waiting and transfer time as gtc-component
  """
  dijkstra_path_length = dict(nx.all_pairs_dijkstra(G, weight = "waiting_time"))
  length = dict()
  path = dict()
  for key, values in dijkstra_path_length.items():
    length[key] = values[0]
    path[key] = values[1]

  """
  The code below calculates the number of transfers made from all to all nodes
  with all_pairs_dijkstra_shortest_path

  """
  df_path = pd.DataFrame(path)
  df_path = df_path.astype('str')
  df_path = df_path.replace('\[','', regex = True, inplace=False)
  df_path = df_path.replace(']','', regex = True, inplace = False)
  count_transfer = []
  for index, row in df_path.iterrows():
    A = row.str.count(',')
    count_transfer.append(A)
  df_transfer = pd.DataFrame(count_transfer)

  """
  The code below, transfers are being penalized with a 300 seconds (5 minutes) penalty
  input: Graph with edge attribute "weight"
  returns: dataframe matrix with count of transfers
  """
  
  transfer_penalty = lambda x: 300*(x-1)
  df_transfer = transfer_penalty(df_transfer)
  df_transfer[df_transfer == (-300)] = 0 
  df_transfer = df_transfer.sort_index(axis=0)
  df_transfer = df_transfer.sort_index(axis=1)
  
  """
  Calculates the travel time consisting of in-vehicle travel time and waiting time
  with all_pairs_dijkstra_shortest_path
  input: Graph with edge attribute "weight"
  returns: dataframe matrix with travel time
  """
  df_waiting = pd.DataFrame(length)             
  df_waiting = df_waiting.sort_index(axis=0)
  df_waiting = df_waiting.sort_index(axis=1)


  """
  The code below calculates the Generalised Travel Costs consisting of in-vehicle travel time,
  waiting time and transfer penalty with all_pairs_dijkstra_shortest_path
  In line with method developed by Luo et al. 2019 (see thesis report for complete
  reference)
  input: Graph with edge attribute "weight"
  returns: dataframe matrix with travel time
  """ 

  df_wandt = df_waiting.add(df_transfer, fill_value=0)
  df_wandt = df_wandt.sort_index(axis=0)
  df_wandt = df_wandt.sort_index(axis=1)

  return df_wandt

In [0]:
rome89_WT = generate_WT_matrix(rome_89_largest_cc)

In [0]:
rome1112_WT = generate_WT_matrix(rome_1112_largest_cc)

In [0]:
rome89_WT.to_csv(DATA_PATH + "/rome/rome89_WT.csv", index = True)
rome1112_WT.to_csv(DATA_PATH + "/rome/rome1112_WT.csv", index = True)

In [0]:
rome1112_WT.to_csv(DATA_PATH + "/rome/rome1112_WT.csv", index = True)

In [0]:
rome89_WT = pd.read_csv(DATA_PATH + "/rome/rome89_WT.csv", sep=",")
rome89_WT = rome89_WT.set_index("Unnamed: 0")
rome89_WT.index.name = None
rome1112_WT = pd.read_csv(DATA_PATH + "/rome/rome1112_WT.csv", sep=",")
rome1112_WT = rome1112_WT.set_index("Unnamed: 0")
rome1112_WT.index.name = None

# Benchmark metric

In [0]:
#With the codes below we select the giant connected component, which also has been selected in the P_space (space of infrastructure) Graph
combined89_L_largest_cc, combined89_smaller_cc_1 = [combined89_L_graph_sn.subgraph(c).copy() for c in nx.connected_components(combined89_L_graph_sn)]
combined1112_L_largest_cc, combined1112_L_smaller_cc_1, combined1112_L_smaller_cc_2 = [combined1112_L_graph_sn.subgraph(c).copy() for c in nx.connected_components(combined1112_L_graph_sn)]

In [0]:
def topological_benchmark_matrix(G):
  """
  Computes the topological shortest path computed in unweighted L_space
  (infrastructural representation). This serves as the benchmark matrix.
  From this matrix the travel impedance per node can be derived. The travel
  impedance is here the number of links traversed in the infrastructural re-
  presentation.
  Input: L_space nx.Graph
  Returns: Pandas Dataframe
  """
  dijkstra_path_length = dict(nx.all_pairs_dijkstra_path_length(G))
  df_length = pd.DataFrame(dijkstra_path_length)
  df_length = df_length.sort_index(axis=0)
  df_length = df_length.sort_index(axis=1)
  return df_length

In [0]:
rome_89_bm = topological_benchmark_matrix(combined89_L_largest_cc)
rome_1112_bm = topological_benchmark_matrix(combined1112_L_largest_cc)

In [0]:
#Saving the dataframes to csv files
rome_1112_bm.to_csv(DATA_PATH + "/rome1112_benchmark.csv", index = True)
rome_89_bm.to_csv(DATA_PATH + "/rome89_benchmark.csv", index = True)

In [0]:
#Loading the data from the csv files into a dataframe
rome_89_bm = pd.read_csv(DATA_PATH + "/rome89_benchmark.csv", sep=",")
rome_89_bm = rome_89_bm.set_index("Unnamed: 0")
rome_89_bm.index.name = None
rome_1112_bm = pd.read_csv(DATA_PATH + "/rome1112_benchmark.csv", sep=",")
rome_1112_bm = rome_1112_bm.set_index("Unnamed: 0")
rome_1112_bm.index.name = None

In [0]:
#Computing the topological travel impedance in L_space infrastructure (benchmark metric)
rome_89_topo_ti = rome_89_bm.sum()/5397
rome_89_topo_ti = pd.DataFrame(rome_89_topo_ti)

In [0]:
rome_1112_topo_ti = rome_1112_bm.sum()/5375
rome_1112_topo_ti = pd.DataFrame(rome_1112_topo_ti)